In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
from tqdm import tqdm
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import matplotlib.pyplot as plt

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# Load Dataset
image_dir = "celebA/celeba/img_align_celeba"
os.makedirs("test", exist_ok=True)
os.makedirs("gan_outputs", exist_ok=True)

transform = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.ImageFolder(root=image_dir, transform=transform)
dataset_size = len(dataset)
indices = list(range(dataset_size))
random.seed(42)
random.shuffle(indices)
split = int(0.1 * dataset_size)
val_indices, train_indices = indices[:split], indices[split:]

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=3000, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=3000, shuffle=False, num_workers=4, pin_memory=True)

/sciclone/data10/iahewababarand/genVision-celebA/gen-env/lib64/python3.11/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
# Define Models
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3, features_g=64):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z_dim, features_g * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_g * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 8, features_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 4, features_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g * 2, features_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_g),
            nn.ReLU(True),
            nn.ConvTranspose2d(features_g, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, img_channels=3, features_d=64):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(img_channels, features_d, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d, features_d * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 2, features_d * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 4, features_d * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_d * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(features_d * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.net(img).view(-1)

In [5]:
# Hyperparameters
z_dim = 100
lr = 2e-4
n_epochs = 50
batch_size = 3000
early_stop_patience = 6

In [6]:
G = Generator(z_dim).to(device)
D = Discriminator().to(device)

In [7]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, z_dim, 1, 1, device=device)

In [8]:
train_g_losses = []
val_g_losses = []
best_val_loss = float('inf')
epochs_no_improve = 0

# Training Loop
for epoch in range(n_epochs):
    g_loss_epoch = 0.0
    d_loss_epoch = 0.0
    num_batches = 0

    G.train()
    for real_imgs, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)

        # Train Discriminator
        z = torch.randn(batch_size, z_dim, 1, 1, device=device)
        fake_imgs = G(z)

        real_labels = torch.ones(batch_size, device=device)
        fake_labels = torch.zeros(batch_size, device=device)

        output_real = D(real_imgs)
        output_fake = D(fake_imgs.detach())
        loss_D = criterion(output_real, real_labels) + criterion(output_fake, fake_labels)

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # Train Generator
        output_fake = D(fake_imgs)
        loss_G = criterion(output_fake, real_labels)

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        g_loss_epoch += loss_G.item()
        d_loss_epoch += loss_D.item()
        num_batches += 1

    # Validation step
    G.eval()
    val_loss = 0.0
    with torch.no_grad():
        for real_imgs, _ in val_loader:
            real_imgs = real_imgs.to(device)
            batch_size = real_imgs.size(0)
            z = torch.randn(batch_size, z_dim, 1, 1, device=device)
            fake_imgs = G(z)
            output_fake = D(fake_imgs)
            loss = criterion(output_fake, torch.ones(batch_size, device=device))
            val_loss += loss.item()

    val_loss /= len(val_loader)
    avg_g_loss = g_loss_epoch / num_batches

    train_g_losses.append(avg_g_loss)
    val_g_losses.append(val_loss)

    with torch.no_grad():
        samples = G(fixed_noise)
        save_image(samples, f"gan_outputs/epoch_{epoch+1:03d}.png", normalize=True)

    print(f"Epoch [{epoch+1}/{n_epochs}]  Loss_D: {d_loss_epoch/num_batches:.4f}  Loss_G: {avg_g_loss:.4f}  Val_Loss_G: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(G.state_dict(), "gan_outputs/best_generator.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stop_patience:
            print("Early stopping triggered!")
            break

Epoch 1/50:   0%|          | 0/61 [00:00<?, ?it/s]

Epoch 1/50:   2%|▏         | 1/61 [00:14<14:40, 14.67s/it]

Epoch 1/50:   3%|▎         | 2/61 [00:15<06:11,  6.30s/it]

Epoch 1/50:   5%|▍         | 3/61 [00:15<03:30,  3.63s/it]

Epoch 1/50:   7%|▋         | 4/61 [00:16<02:15,  2.37s/it]

...

Epoch 4/50:  67%|██████▋   | 41/61 [01:47<01:12,  3.62s/it]

Epoch 4/50:  69%|██████▉   | 42/61 [01:47<00:50,  2.67s/it]

Epoch 4/50:  70%|███████   | 43/61 [01:47<00:36,  2.00s/it]

Epoch 4/50:  72%|███████▏  | 44/61 [01:48<00:26,  1.53s/it]

Epoch 4/50:  74%|███████▍  | 45/61 [01:56<00:57,  3.60s/it]

In [ ]:
# Plot losses
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(train_g_losses) + 1), train_g_losses, label='Training Generator Loss')
plt.plot(range(1, len(val_g_losses) + 1), val_g_losses, label='Validation Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Generator Training vs Validation Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("gan_outputs/loss_plot.png")
plt.show()

In [ ]:
# Generate final images using best generator
os.makedirs("gan_outputs/generated", exist_ok=True)
G.load_state_dict(torch.load("gan_outputs/best_generator.pth"))
G.eval()
with torch.no_grad():
    for i in tqdm(range(0, 10000, 64), desc="Generating final images"):
        z = torch.randn(64, z_dim, 1, 1, device=device)
        gen_imgs = G(z)
        for j in range(gen_imgs.size(0)):
            save_image(gen_imgs[j], f"gan_outputs/generated/{i + j:05d}.png", normalize=True)